In [1]:
import numpy as np
from scipy import linalg as lg
import time

In [2]:
def use_rank(s, eps):
    eps = eps**2
    k = 1
    spec = np.sum(s**2)
    while k < s.shape[0] and np.sum(s[k:]**2) >= eps:
        k = k + 1
    return k, np.sqrt(np.sum(s[k:]**2))

In [3]:
def st_HOSVD(T, eps=10**(-8), ranks=None):
    factors = []
    dims = list(T.shape)
    d = len(dims)

    for i in range(d):
        permute = np.array([i])
        permute = np.append(permute, np.arange(i))
        permute = np.append(permute, np.arange(i+1, d))

        T = np.transpose(T, permute)
        T = T.reshape((dims[i], -1))

        U, s, Vh = np.linalg.svd(T, full_matrices=False)

        if ranks:
            rank = ranks[i]
        else:
            rank, m_eps = use_rank(s, eps / np.sqrt(d - i))
            eps = np.sqrt(eps**2 - m_eps**2)

        factors.append(U[:, :rank])

        T = s[:rank].reshape((-1, 1)) * Vh[:rank, :]

        dims[i] = rank

        now_dim = np.array(dims[i]).astype(int)
        now_dim = np.append(now_dim, np.array(dims[:i])).astype(int)
        now_dim = np.append(now_dim, np.array(dims[i+1:])).astype(int)


        T = T.reshape(now_dim)

        permute = np.arange(1, i+1)
        permute = np.append(permute, 0)
        permute = np.append(permute, np.arange(i+1, d))

        T = np.transpose(T, permute)

    return T, factors, dims

# Cteate tensor

In [4]:
sizes = np.array((10, 20, 30, 40))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            for l in range(sizes[3]):
                T[i, j, k, l] = np.sin(i * j * k * l)

# Run st-HOSVD

In [41]:
tensor, factors, ranks = st_HOSVD(T.copy(), eps=10**(-10), ranks=None)

In [42]:
ranks

[9, 19, 29, 39]

In [43]:
d = len(ranks)

for i in range(d):
    permute = np.array([i])
    permute = np.append(permute, np.arange(i))
    permute = np.append(permute, np.arange(i+1, d))

    tensor = np.transpose(tensor, permute)
    tensor = tensor.reshape((ranks[i], -1))

    tensor = factors[i] @ tensor

    ranks[i] = sizes[i]

    now_dim = np.array(ranks[i]).astype(int)
    now_dim = np.append(now_dim, np.array(ranks[:i])).astype(int)
    now_dim = np.append(now_dim, np.array(ranks[i+1:])).astype(int)

    tensor = tensor.reshape(now_dim)

    permute = np.arange(1, i+1)
    permute = np.append(permute, 0)
    permute = np.append(permute, np.arange(i+1, d))

    tensor = np.transpose(tensor, permute)

In [44]:
print(np.linalg.norm((tensor - T).reshape(-1,)) / np.linalg.norm(T.reshape(-1,)))

3.3757365816872955e-15
